# 空間を扱うプログラミング

## §1 座標系 (coordinate system)

ここでは直交座標系 (座標軸が互いに直行) を前提とする。

* 3Dグラフィックスでは大きく4つの座標系が存在する
  1. ワールド座標系
     * 3D空間全体に対しただ一つ存在する絶対座標を表す座標系
     * ここでは座標軸名は `x` `y` `z` と小文字で表記する
  2. オブジェクトのローカル座標系
     * ワールド上の個々のオブジェクトに対し定義される相対座標を表す座標系
     * ワールド内でオブジェクトを移動させたり回転させたりすると、ワールド原点に対するローカル座標系の位置、向きも変化する
     * オブジェクト視点では、ローカル座標系は常に変化しないように見える
     * ここでは座標軸名は `X` `Y` `Z` と大文字で表記する
  3. カメラ座標系
     * カメラのイメージセンサーを基準とした座標系
     * カメラにとってのローカル座標系でもある
     * カメラの視線方向・イメージセンサーの縦方向・イメージセンサーの横方向を定義する
     * カメラの位置・姿勢を、カメラから見たワールド座標軸の位置・向きで表現する
     * ここでは座標軸名は `X` `Y` `Z` と大文字で表記する (ローカル座標系と同様に扱う)
  4. 画像座標系
     * カメラ画像上の座標を定義する座標系
     * ここでは座標軸名は `X` `Y` `Z` と大文字で表記する (ローカル座標系と同様に扱う)
     * OpenCV では、画像の左上隅を原点とし、右方向が `+X`、下方向が `+Y`


### 右手系 (right-handed system)・左手系 (left-handed system)

* 3D座標系を定義するとき、座標軸の向きには大きく2通りの決め方がある: 右手系と左手系
* ここで両手を出して、それぞれの手で親指、人差し指、中指が直行するようにしてみよう
  * 親指、人差し指、中指の向きがそれぞれ `+x`、`+y`、`+z` 方向に対応するものと考えてほしい
  * 右手で定義された座標軸と、左手で定義された座標軸は決してぴったり重なり合わないことがわかるだろう

<img alt="Right-handed and left-handed coordinate systems" src="./notebook_assets/images/1_right-left-handed-corrdinate-system.png" style="width:600px;" />

* ややこしいことに、ツールやライブラリによって右手系、左手系が異なることがある
  * もっというと、同じ右手系でも空間に対して座標軸の向きの割り当て方が違っていたりもするので、さらにややこしい
  * 右手系の例
    * Open3D
    * OpenCV
    * ARKit
  * 左手系の例
    * Unity
    * Unreal Engine


### 演習環境

* この演習では内部的に python パッケージの一つである [Open3D](https://www.open3d.org/docs/release/) を利用している
  * Open3D には点群・メッシュといった基本的な 3D オブジェクトの表示、演算を行う API が実装されている
  * ただし演習ではオブジェクトの位置・姿勢表現の理解に注力することを意図しており、 Open3D の API を直接操作することはない
* このテキストブックでは、下図のような座標軸とカメラオブジェクトを含むワールドを使用する
  * 赤・緑・青の点のセットがワールド座標軸の伸びる方向を示している
    * 点は 0.5 m 間隔で配置されている
    * また、直行する大きな赤・緑・青の矢印がワールド座標軸のそれぞれ `+x`, `+y`, `+z` 方向を示している
  * ワールドには他に地面を表現した平面と、カメラを表すオブジェクトが存在する
    * 床は `xz` 平面に並行で、`y` 座標が -1 m の領域に広がっている
    * カメラはレンズの中心がちょうどワールド座標原点に一致しており、レンズが `+y` 方向に向いている
      * カメラのローカル座標軸と、カメラ座標系の座標軸が一致するようにしている
      * カメラにくっついている小さな矢印のセットはローカル座標系=カメラ座標系の座標軸を表しており、赤・緑・青がそれぞれ `+X`, `+Y`, `+Z` 方向を表す
        * カメラのレンズの向きが `+Z`、カメラで撮られた写真の右方向が `+X`、下方向が `+Y` になる

   <img alt="Initial world setting" src="./notebook_assets/images/1_initial_world.png" style="width:600px;"/>


In [ ]:
# 必要なライブラリをインポート
from __future__ import annotations

import numpy as np
import open3d as o3d
import sympy as sp
from IPython.display import Math, display
from scipy.spatial.transform import Rotation, Slerp

from util_lib.transformable_object import TransformableObject
from util_lib.types import EulerOrder, Transform
from util_lib.visualization import draw_geometries
from util_lib.world import create_coordinate_objects

# ワールド・カメラ座標系の作成と読み込み
world_coordinate: o3d.geometry.Geometry = create_coordinate_objects()
original_camera = TransformableObject.load_model("./data/camera.gltf", enable_post_processing=True)

# 行列表示用の関数
def print_matrix(mat: np.ndarray | Transform) -> None:
    if isinstance(mat, Transform):
        mat_sym = sp.Matrix(mat.get_matrix())
    else:
        mat_sym = sp.Matrix(mat)
    mat_sym = mat_sym.applyfunc(lambda x: sp.Symbol(f"{x:.3f}"))
    display(Math(sp.latex(mat_sym)))


#### [課題1] ワールドの確認

* (1-1) プリセットとして用意されたワールドを表示しよう
* (1-2) ワールドの広がりに対するワールド座標軸、カメラのローカル座標軸の向きを確認しよう
  * ワールド座標軸の向きの関係を見て、右手系であることを確認してください
  * グレーの平面を水平な地面とみなしたとき、ワールド座標軸が向いている方向を確認してください
    * 上下・左右・前後と座標軸の向きの関係を見てください
* 確認ができたら次のステップに進むため、Open3D ビューワーを閉じてください (ビュ-ワーの❌️ボタンを押すか、ビュワーウィンドウがアクティブな状態で `ESC` キーを押す)

(参考) Open3D ビューワー操作
* 左クリックでドラッグ: 注視点まわりにカメラを回転
* スクロール: 中心点にカメラを近づける or 遠ざける
* Ctrl+左ドラッグ: 注視点を移動
* Shift+左ドラッグ: 視線方向に対しカメラを回転

In [ ]:
# ワールド座標系とカメラの初期位値を表示
#
# 下記のような警告は問題ないので無視してください。
# [Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display

draw_geometries(world_coordinate, original_camera)

## §2 ワールド座標に対するオブジェクトの操作

* オブジェクトに対して定義される操作には下記がある
  * 並進 (translate, translation)
  * 回転 (rotate, rotation)
  * 拡大縮小 (scale, scaling)
    * 鏡映、反転を含む (負の拡大率を定義する)
* これらの操作をまとめて transform、transformation と呼ぶ

オブジェクトの回転・並進の例

* ワールドの +y 方向にあるカメラが回転・並進変換後のオブジェクト
  1. ワールド x 軸周りに90度回転
  2. ワールド y 軸方向に+1並進

<img alt="Example of object transformation" src="./notebook_assets/images/1_example_transformation.png" style="width:600px;"/>

In [ ]:
# Just example

# オブジェクトを新規にコピー
camera = original_camera.copy()

# コピーしたオブジェクトを変換
camera.translate((0, 1, 0)) # y軸方向に1m移動
camera.rotate_by_euler(EulerOrder.xyz, (90, 0, 0)) # x軸周りに90度回転

draw_geometries(world_coordinate, original_camera, camera)

## §2.1 オブジェクトの並進

* 並進は、3次元の並進ベクトル `(dx, dy, dz)` で表現する
  * 並進操作により、オブジェクトの位置ベクトルに並進ベクトルが足し合わされる

#### [課題2] オブジェクトを並進してみよう

* (2-1) カメラオブジェクトを `y` 方向に `+1` 平行移動する
* (2-2) カメラオブジェクトを `x` 方向に `+1` 平行移動する、続いてカメラオブジェクトを `z` 方向に `-1` 平行移動する
* (2-3) カメラオブジェクトを1回の並進操作で (2-2) の最終状態に移動する (並進操作の合成)

In [ ]:
# (2-1)
camera = original_camera.copy()
camera.translate() # implement here
draw_geometries(world_coordinate, original_camera, camera)

In [ ]:
# (2-2)
camera_2_2 = original_camera.copy()
camera_2_2.translate() # implement here
camera_2_2.translate() # implement here
draw_geometries(world_coordinate, original_camera, camera_2_2)

In [ ]:
# (2-3)
camera_2_3 = original_camera.copy()
camera_2_3.translate() # implement here
draw_geometries(world_coordinate, original_camera, camera_2_2, camera_2_3)

## §2.2 オブジェクトの姿勢回転

* 3D空間において回転を表現するのはやや複雑である
* 代表的な回転表現には下記がある
  * Euler (オイラー) 角
  * 回転行列
  * クォータニオン
  * 回転ベクトル

### §2.2.1 Euler 角
* オブジェクトの姿勢を、座標軸周りの回転を合成として表現する
* Euler 角の角度の単位
  * 度数法の場合と弧度法 (ラジアン) の場合がある
  * このチュートリアルではデフォルトで度数法を使うようになっている
* Euler 角の符号
  * 右手系では「右ネジの法則」に従う
    * 右ネジが座標軸の正の方向に進むとき、その回転方向を正とする
    * または右手で「サムアップ」し、親指を座標軸の正方向に向ける。このとき他の指を握る方向が回転の正である
* ex. ワールド座標軸に対しまず x 軸周りに 40° 回転、次に y 軸周りに 30° 回転、最後に z 軸周りに 30° 回転する
  * → Euler 角は回転軸と回転順序がワールド座標系の `xyz`、回転角の組は `(rx, ry, rz) = (40, 30, 30)` と表せる

<img alt="Synthesis of Euler angles" src="./notebook_assets/images/1_synthesis_of_euler_rotation.png" style="width:400px;" />


In [ ]:
# 3軸の回転を一度に与える (回転順序はワールド座標軸の x → y → z)
camera_final = original_camera.copy()
camera_final.translate((1, 1.5, 0))
camera_final.rotate_by_euler(EulerOrder.xyz, (40, 30, 30))

# x → y → zと1軸ずつ回転する
camera_x_rotated = original_camera.copy()
camera_x_rotated.translate((0, 0.5, 0))
camera_x_rotated.rotate_by_euler(EulerOrder.xyz, (40, 0, 0))

camera_xy_rotated = camera_x_rotated.copy()
camera_xy_rotated.translate((0, 0.5, 0))
camera_xy_rotated.rotate_by_euler(EulerOrder.xyz, (0, 30, 0))

camera_xyz_rotated = camera_xy_rotated.copy()
camera_xyz_rotated.translate((0, 0.5, 0))
camera_xyz_rotated.rotate_by_euler(EulerOrder.xyz, (0, 0, 30))

# 最終的な姿勢は同じ
draw_geometries(world_coordinate, original_camera, camera_final,
                camera_x_rotated, camera_xy_rotated, camera_xyz_rotated)

### 回転操作の非可換性

回転の順序が変わると、最終的な姿勢が変わってしまうことに注意

* ex. ワールド座標軸の x、y、z 軸まわりにそれぞれ 40°、30°、30° 回転することを考える
* このとき x → y → z の順に回転するのと (下図左)、z → y → x の順に回転するのと (下図右) では結果が異なる


<img alt="Non-commutativity of rotations" src="./notebook_assets/images/1_non_commutativity_of_rotation.png" style="width:600px;" />

In [ ]:
# 3軸の回転を与える (回転順序はワールド座標軸の x → y → z)
camera_rotate_xyz = original_camera.copy()
camera_rotate_xyz.translate((-0.5, 1, 0))
camera_rotate_xyz.rotate_by_euler(EulerOrder.xyz, (40, 30, 30))

# 3軸の回転を与える (回転順序はワールド座標軸の z → y → x)
camera_rotate_zyx = original_camera.copy()
camera_rotate_zyx.translate((0.5, 1, 0))
camera_rotate_zyx.rotate_by_euler(EulerOrder.zyx, (30, 30, 40))

draw_geometries(world_coordinate, original_camera, camera_rotate_xyz, camera_rotate_zyx)

#### [課題3] オブジェクトをオイラー角で回転してみよう

- (3-1) カメラをワールド座標軸の z → y → x の順にそれぞれ 45°、90°、-45°回転しよう
  - まず、最終的にどのような姿勢になるか考えてください
  - カメラを回転させて、事前の想定とあっているか確認してください

In [ ]:
# (3-1)
camera = original_camera.copy()
camera.translate((-0.5, 1.5, 0))
camera.rotate_by_euler() # implement here

# 1軸ずつ回転していく様子を確認しよう
camera_z = original_camera.copy()
camera_z.translate((0.5, 0.5, 0))
camera_z.rotate_by_euler() # implement here

camera_zy = camera_z.copy()
camera_zy.translate((0, 0.5, 0))
camera_zy.rotate_by_euler() # implement here

camera_zyx = camera_zy.copy()
camera_zyx.translate((0, 0.5, 0))
camera_zyx.rotate_by_euler() # implement here

draw_geometries(world_coordinate, original_camera, camera, camera_z, camera_zy, camera_zyx)

### (参考) 外因性 (extrinsic) 回転と内因性 (intrinsic) 回転

* 外因性回転とは、ワールド座標軸に対する回転のこと
* 内因性回転とは、ローカル座標軸に対する回転のこと
* 内因性回転の場合、オブジェクトの回転に伴いその後の回転軸も回転していることになる
* このチュートリアルでは `EulerOrder` でオイラー角の回転順序を表現しているが、ここでは `xyz` のような小文字が外因性の回転順序を、`XYZ` のような大文字が内因性の回転順序を表している
* 回転の角度、回転順序が同じでも、外因性回転 (下図左) と内因性回転 (下図右) では結果が異なる

<img alt="Extrinsic and intrinsic rotation" src="./notebook_assets/images/1_extrinsic_and_intrinsic_rotation.png" style="width:600px;" />

### §2.2.2 回転行列

* オブジェクトの回転を行列で表現する
* 3次元空間内の回転は 3x3 行列で表現でき、そのような行列を回転行列と呼ぶ
  * 回転行列では、1軸分の回転を表現することも、3軸分の回転を表現することもできる
* 回転操作の合成は、回転行列の行列積で表現される
  * ex. 回転操作1~3をそれぞれ回転行列 $\mathbf{R}_1$、$\mathbf{R}_2$、$\mathbf{R}_3$ と表す。またオブジェクトの座標の集合を $\mathbf{x}$ と表す
  * 回転操作を $\mathbf{R}_1$ → $\mathbf{R}_2$ → $\mathbf{R}_3$ の順に適用することは、 $\mathbf{R}_3 \mathbf{R}_2  \mathbf{R}_1 \mathbf{x}$ という順番で行列を掛けることに対応する

回転行列の生成
* ここでは Euler 角から回転行列を計算する
* python では、[`scipy` パッケージの `scipy.spatial.transform.Rotation`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.html) を使うと良い
* 例として、ワールド座標軸 x → y → z の順に 40°、30°、30° の回転を考える

回転行列の特徴
* Euler 角に対し、回転順序や外因性・内因性による混乱を避けることができる
* Euler 角に対し、回転の合成が行列積で表現でき、数学的に扱いやすい
* Euler 角に比べるとヒューマンリーダビリティで劣る
* Euler 角に比べて回転表現に必要なパラメータ数が9つも必要で冗長
* 後の章で触れるが、行列表現では並進変換を含めて統一的に扱うことができる

In [ ]:
# 回転しない場合の回転行列は単位行列に等しい
identity_matrix = Rotation.from_euler(EulerOrder.xyz, (0, 0, 0), degrees=True).as_matrix()
print("Identity matrix")
print_matrix(identity_matrix)

# x軸周りに40度、y軸周りに30度、z軸周りに30度回転する場合の回転行列
rotate_matrix = Rotation.from_euler(EulerOrder.xyz, (40, 30, 30), degrees=True).as_matrix()
print("3軸回転行列")
print_matrix(rotate_matrix)

# 1軸ずつ回転する場合の回転行列
rotate_matrix_x = Rotation.from_euler(EulerOrder.xyz, (40, 0, 0), degrees=True).as_matrix()
rotate_matrix_y = Rotation.from_euler(EulerOrder.xyz, (0, 30, 0), degrees=True).as_matrix()
rotate_matrix_z = Rotation.from_euler(EulerOrder.xyz, (0, 0, 30), degrees=True).as_matrix()
print("1軸ずつ回転した場合の回転行列")
print_matrix(rotate_matrix_x)
print_matrix(rotate_matrix_y)
print_matrix(rotate_matrix_z)

# 1軸ずつ回転した場合の回転行列をまとめる
rotate_matrix_xyz = rotate_matrix_z @ rotate_matrix_y @ rotate_matrix_x
print("1軸ずつの回転を合成した回転行列")
print_matrix(rotate_matrix_xyz)

# 2通りで計算した行列が等しいことの確認 (等しくないと例外が発生する)
np.testing.assert_array_almost_equal(rotate_matrix, rotate_matrix_xyz)

In [ ]:
# オイラー角による回転
camera_by_euler = original_camera.copy()
camera_by_euler.translate((-0.5, 1, 0))
camera_by_euler.rotate_by_euler(EulerOrder.xyz, (40, 30, 30))

# 回転行列による回転
rotate_matrix = [
    [0.750, -0.105, 0.653],
    [0.433, 0.824, -0.365],
    [-0.500, 0.557, 0.663],
]

camera_by_matrix = original_camera.copy()
camera_by_matrix.translate((0.5, 1, 0))
camera_by_matrix.rotate(rotate_matrix)

draw_geometries(world_coordinate, original_camera, camera_by_euler, camera_by_matrix)

### §2.2.3 クォータニオン (Quaternion)

* Euler 角・回転行列の課題
  * 回転行列は Euler 角に比べてパラメータが冗長
  * Euler 角・回転行列ともに、2つの姿勢の間を補間するような回転表現を得るのが面倒
* クォータニオンは3次元空間での回転を4つのパラメータ (qw, qx , qy, qz) で表現する
* クォータニオンでは回転行列と同様、回転順序や外因性・内因性の曖昧性を避けることができる
* 2つのクォータニオンの中間状態を計算することが簡単

クォータニオンの計算
* [`scipy` パッケージの `scipy.spatial.transform.Rotation`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.html)を使うと良い
  * このパッケージでは、クォータニオンの要素の順番が (qx, qy, qz, qw) であることに注意

In [ ]:
# 回転しない場合のクォータニオンは qw だけ1で他が0
identity_quaternion = np.array([0, 0, 0, 1]) # (qx, qy, qz, qw)
np.testing.assert_array_almost_equal(
    identity_quaternion,
    Rotation.from_euler(EulerOrder.xyz, (0, 0, 0), degrees=True).as_quat(),
)

# x軸周りに40度、y軸周りに30度、z軸周りに30度回転する場合のクォータニオン
rotate_quaternion = Rotation.from_euler(EulerOrder.xyz, (40, 30, 30), degrees=True).as_quat()
print_matrix(rotate_quaternion)

In [ ]:
# Euler 角による回転
camera_by_euler = original_camera.copy()
camera_by_euler.translate((-0.5, 1, 0))
camera_by_euler.rotate_by_euler(EulerOrder.xyz, (40, 30, 30))

# クォータニオンによる回転
rotate_quaternion_xyzw = np.array([0.256, 0.320, 0.149, 0.900])
camera_by_quaternion = original_camera.copy()
camera_by_quaternion.translate((0.5, 1, 0))
camera_by_quaternion.rotate_by_quaternion(rotate_quaternion_xyzw)

draw_geometries(world_coordinate, original_camera, camera_by_euler, camera_by_quaternion)

### (参考) 回転の補間

* ここでは回転順序 `xyz` の Euler 角 (0, 0, 0) ~ (40°, 30°, 30°) で表される回転を補間してみる
* 自前で実装した線形補間と、[scipy で提供されている球面線形補間 (SLERP)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Slerp.html0) によりクォータニオンを補間する
* 2つのクォータニオン $\mathbf{q}_1$ と $\mathbf{q}_2$ を線形補間するとは、$0 <= r <=1$ である $r$ を用いて下記を計算すること
  * $\mathbf{q}(r) = (1-r) \mathbf{q}_1 + r \mathbf{q}_2$
  * クォータニオンをベクトルとみなすと、ベクトルの加重平均を取ることに相当する
* なお、クォータニオンの補間を行う場合、球面線形補間のほうが自然とされる

下図では真ん中の最終的なオブジェクト姿勢をはさんで、左が自前実装した線形補間の結果、右が球面線形補間の結果である。

<img alt="Interpolating rotations" src="./notebook_assets/images/1_interpolating_rotation.png" style="width:600px;" />

In [ ]:
class LinearInterpolation:

    """scipy.spatial.transform.Slerp と同じインターフェースを持つ線形補間クラス."""

    def __init__(self, key_times, key_values):
        self.__key_times = key_times
        self.__key_values = key_values

    def __call__(self, time):
        return self.__interpolate(time)

    def __interpolate(self, time):
        if time <= self.__key_times[0]:
            return self.__key_values[0]
        if time >= self.__key_times[-1]:
            return self.__key_values[-1]

        for i in range(len(self.__key_times) - 1):
            if self.__key_times[i] <= time < self.__key_times[i + 1]:
                t = (time - self.__key_times[i]) / (self.__key_times[i + 1] - self.__key_times[i])
                return self.__key_values[i] * (1 - t) + self.__key_values[i + 1] * t

        raise ValueError("Invalid time")

In [ ]:
key_times = [0, 1]
key_euler_rotations = [(0, 0, 0), (40, 30, 30)]
rotations = Rotation.from_euler(EulerOrder.xyz, key_euler_rotations, degrees=True)

# 補間クラスのインスタンス生成
linear = LinearInterpolation(key_times, [r.as_quat() for r in rotations])
slerp = Slerp(key_times, rotations)


final_camera = original_camera.copy()
final_camera.translate((0, 1.5, 0))
final_camera.rotate_by_euler(EulerOrder.xyz, key_euler_rotations[1])

print("線形補間")
cameras_linear = []
for time in np.arange(0, 1.1, 0.1):
    interpolated_quat = linear(time)
    print(f"Time: {time:.1f}, Quaternion: {interpolated_quat}")

    camera_by_linear = original_camera.copy()
    y_pos = time * 1.5
    camera_by_linear.translate((-1, y_pos, 0))
    camera_by_linear.rotate_by_quaternion(interpolated_quat)
    cameras_linear.append(camera_by_linear)

print("球面線形補間")
cameras_slerp = []
for time in np.arange(0, 1.1, 0.1):
    interpolated_quat = slerp(time).as_quat()
    print(f"Time: {time:.1f}, Quaternion: {interpolated_quat}")

    camera_by_slerp = original_camera.copy()
    y_pos = time * 1.5
    camera_by_slerp.translate((1, y_pos, 0))
    camera_by_slerp.rotate_by_quaternion(interpolated_quat)
    cameras_slerp.append(camera_by_slerp)
draw_geometries(world_coordinate, *cameras_linear, *cameras_slerp, final_camera)

## §2.3 Transformation

§2.1、§2.2 でそれぞれ3次元空間でのオブジェクトの並進、回転の表現を説明してきた。

ここでは、並進と回転を統一して扱う表現方法として、Transformation を扱う。

Transformation は 4x4 の行列である。
* 並進を表すベクトルを $[t_x, t_y, tz]$ とする
* 回転を表す行列を $\mathbf{R}$ とし、その成分を $r_{11}$ ~ $r_{33}$ で表す
* このとき Transformation 行列 $\mathbf{T}$ は下記のように表すことができる

$$
\mathbf{T} = \begin{bmatrix}
r_{11} & r_{12} & r_{13} & t_x \\
r_{21} & r_{22} & r_{23} & t_y \\
r_{31} & r_{32} & r_{33} & t_z \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

* Transformation 行列を掛けることで、並進・回転を同時に適用することができる
  * このため、ある時点のオブジェクトの位置・姿勢を表現するためにしばしば用いられる
* 複数の Transformation $\mathbf{T}_1$ → $\mathbf{T}_2$ → $\mathbf{T}_3$ を連続で適用することは下記のような行列積で表現できる
  * $\mathbf{T}_3 \mathbf{T}_2 \mathbf{T}_1 \mathbf{x}$


In [ ]:
camera = original_camera.copy()
camera.translate((-1, 1.5, 1))
camera.rotate_by_euler(EulerOrder.xyz, (40, 30, 30))

camera_transformed = original_camera.copy()
rotation_matrix = Rotation.from_euler(EulerOrder.xyz, (40, 30, 30), degrees=True).as_matrix()

print("回転行列")
print_matrix(rotation_matrix)
transformation = Transform([
    [ 0.750, -0.105,  0.653,  -1],
    [ 0.433,  0.824, -0.365, 1.5],
    [-0.500,  0.557,  0.663,   1],
    [     0,      0,      0,   1],
])

camera_transformed.transform(transformation)

draw_geometries(world_coordinate, original_camera, camera, camera_transformed)

* Transformation 行列の逆行列をかけることにより、変換前のオブジェクトの位置姿勢を復元できる
  * $\mathbf{T}^{-1} \mathbf{T} \mathbf{x} = \mathbf{I} \mathbf{x} = \mathbf{x}$

In [ ]:
transformation = Transform([
    [ 0.750, -0.105,  0.653,  -1],
    [ 0.433,  0.824, -0.365, 1.5],
    [-0.500,  0.557,  0.663,   1],
    [     0,      0,      0,   1],
])

inverse_transformation = transformation.inv()
np.testing.assert_array_almost_equal((inverse_transformation @ transformation).get_matrix(), np.identity(4))

camera_transformed = original_camera.copy()
camera_transformed.transform(transformation)
camera_transformed.transform(inverse_transformation)

draw_geometries(world_coordinate, original_camera, camera_transformed)

### 環境による座標系の違い

複数のプラットフォーム・ツールを連携する場合、それぞれの座標系の違いに気をつける必要がある。

プラットフォーム・ツールごとに座標軸の向きや長さの単位が異なる場合があり、
座標系間の差分を吸収しなければ、オブジェクトの位置や姿勢が期待と異なってしまうことに繋がりかねない。

ここでは例として、Open3D と Unity 上で同じオブジェクトを表示してみて、それぞれの環境の座標軸との関係を比べてみよう。

* Unity hub を起動し、`New project` で新しいプロジェクトを作成
* Unity エディタの `Assets` 部分に [`data/camera_for_unity/`](./data/camera_for_unity/camera.gltf.obj) ディレクトリをドラッグアンドドロップ

<img alt="Import GLTF object into Unity project" src="./notebook_assets/images/1_import_gltf_to_unity.png" style="width:800px;" />

* インポートされた `camera_for_unity` の中から `camera.gltf` をドラッグアンドドロップ

<img alt="Locate GLTF object into Unity project" src="./notebook_assets/images/1_locate_gltf_in_unity_space.png" style="width:800px;" />


#### [課題4] Open3D と Unity でワールド座標系がどのように違うか確認しよう

* (4-1) Unity は右手系か、左手系か?
* (4-2) Open3D と Unity でワールド座標系の向きはどのように違うか?